In [1]:
#installing dependencies
!pip install geocoder
!pip install folium

In [2]:
import pandas as pd
import numpy as np
import requests
import geocoder
import folium


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extract_data = requests.get(url)

df = pd.read_html(extract_data.content,header=0,na_values=['Not assigned'])[0]
df.dropna(subset=['Borough'],inplace = True)
df.reset_index(drop=True,inplace=True)
df.head()



,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

In [5]:
import geocoder
location = 'http://cocl.us/Geospatial_data'
df_geoloc = pd.read_csv(location)
df = df.join(df_geoloc.set_index('Postal Code'),on='Postal Code')

df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
df.sort_values('Borough',inplace=True)
df.reset_index(drop=True,inplace=True)
df.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
1,M4S,Central Toronto,Davisville,43.704324,-79.388790
2,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [7]:
borough_name = df['Borough'].unique().tolist()
borough_name
lat_data = []
lng_data=[]

In [8]:
for i in borough_name:
    df_area = df[df['Borough'].str.contains(i)]
    lat_data.append(df_area['Latitude'].mean())
    lng_data.append(df_area['Longitude'].mean())

print('Latitude shape: ',len(lat_data))
print('Longitude shape:', len(lng_data))
print('borough shape:',len(borough_name))


Latitude shape:  10
Longitude shape: 10
borough shape: 10


In [9]:
borough_color={}
for b in borough_name:
    borough_color[b] = '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [10]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
lat_n1 = location.latitude
long_n1 = location.longitude
print(lat_n1, long_n1)

map_toronto = folium.Map(location=[lat_n1, long_n1], zoom_start=10)

for lat, long, borough in zip(lat_data,lng_data,borough_name):
    label_text = borough
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='black',#borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)

map_toronto

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


43.6534817 -79.3839347


In [11]:
CLIENT_ID = 'UBWZ0SWXLKGLRO5GI3LF0DL1C3JITRFCX13JPNYMJREIV00L'
CLIENT_SECRET = '3VC4JWGDHAEXS1W4QJWXK4CBPAIN5FWE0NF1MG4OSMWCPTNI'
VERSION = '20200716'
LIMIT = 300
RADIUS = 1000

In [12]:
results = [0]

In [13]:
for i,lat,lng in zip(borough_name,lat_data,lng_data):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    RADIUS, 
    LIMIT)
    results.append(requests.get(url).json())    

In [20]:
from pandas.io.json import json_normalize

venues=results[1]['response']['groups'][0]['items']
newnearby_venues = json_normalize(venues)
print('In the Borough {}, we have this entries {}'.format(borough_name[0],newnearby_venues.shape))

for i in range(2,len(results)):
    venues = results[i]['response']['groups'][0]['items']
    newvenues = json_normalize(venues)
    newnearby_venues = pd.concat([newnearby_venues,newvenues],axis=0,join='outer',sort=True)
    print('In the Borough {}, we have this entries {}'.format(borough_name[i-1],newvenues.shape))

In the Borough Central Toronto, we have this entries (100, 22)
In the Borough Downtown Toronto, we have this entries (100, 22)
In the Borough East Toronto, we have this entries (87, 22)
In the Borough East York, we have this entries (20, 22)
In the Borough Etobicoke, we have this entries (9, 20)
In the Borough Mississauga, we have this entries (53, 21)
In the Borough North York, we have this entries (13, 20)
In the Borough Scarborough, we have this entries (16, 20)
In the Borough West Toronto, we have this entries (100, 22)
In the Borough York, we have this entries (47, 22)


In [15]:
newnearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

filteredcolumn = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = newnearby_venues.loc[:,filteredcolumn]
nearby_venues.head()

nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.columns = [col.split(',')[-1] for col in nearby_venues.columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)
nearby_venues['categories'] = nearby_venues['venue.categories']

filteredcolumn = ['name','categories','lat','lng']
nearby_venues = nearby_venues.loc[:,filteredcolumn]


nearby_venues.head()


,name,categories,lat,lng
0,Balsamico,Italian Restaurant,43.701505,-79.397162
1,Little Sister,Indonesian Restaurant,43.701552,-79.397163
2,La Salumeria,Deli / Bodega,43.701255,-79.397117
3,Five Doors North,Italian Restaurant,43.702236,-79.397526
4,Noorden Food x Bar,Restaurant,43.702814,-79.397645


In [17]:
nearby_venues['categories'].unique().tolist()

['Italian Restaurant',
 'Indonesian Restaurant',
 'Deli / Bodega',
 'Restaurant',
 'Middle Eastern Restaurant',
 'Bar',
 'Plaza',
 'Sushi Restaurant',
 'Café',
 'Bookstore',
 'Syrian Restaurant',
 'Coffee Shop',
 'Park',
 'Yoga Studio',
 'Supermarket',
 'Gym',
 'Vegetarian / Vegan Restaurant',
 'French Restaurant',
 'Ramen Restaurant',
 'Dessert Shop',
 'Arts & Crafts Store',
 'Tea Room',
 'Movie Theater',
 'Taco Place',
 'Salad Place',
 'Vietnamese Restaurant',
 'Gastropub',
 'Burger Joint',
 'Japanese Restaurant',
 'Wine Bar',
 'Breakfast Spot',
 'Indian Restaurant',
 'Ice Cream Shop',
 'Mexican Restaurant',
 'Caribbean Restaurant',
 'Trail',
 'Bakery',
 'Pub',
 'Pizza Place',
 'Food & Drink Shop',
 'Seafood Restaurant',
 'Skating Rink',
 'Convenience Store',
 'Electronics Store',
 'Bank',
 'Fast Food Restaurant',
 'Bubble Tea Shop',
 'Persian Restaurant',
 'Gym / Fitness Center',
 'Gaming Cafe',
 'Brewery',
 'Tennis Court',
 'Juice Bar',
 'Sandwich Place',
 'Thai Restaurant',
 'Hote

In [18]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
lat_n1 = location.latitude
long_n1 = location.longitude
print(lat_n1, long_n1)

map_toronto = folium.Map(location=[lat_n1, long_n1], zoom_start=10)

for lat, long, category, name in zip(nearby_venues['lat'],nearby_venues['lng'],nearby_venues['categories'],nearby_venues['name']):
    label_text = name + ' - ' + category
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='black',#borough_color[borough],
        fill_color='red',#borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)

map_toronto

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


43.6534817 -79.3839347
